In [ ]:
pip install finance-datareader

In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
import datetime

In [ ]:
#미국 주식 정보를 가져온다
#  KRX   : KRX 전체 종목
#  KOSPI : KOSPI 종목
#  KOSDAQ: KOSDAQ 종목
#  KONEX : KONEX 종목

#  NASDAQ: 나스닥 종목
#  NYSE  : 뉴욕증권거래소 종목

df_nyse = fdr.StockListing('NYSE')
df_nasdaq = fdr.StockListing('NASDAQ')

100%|██████████| 4247/4247 [00:17<00:00, 241.95it/s]


In [ ]:
#NYSE뉴욕증권거래소 주식종목불러오기
df_nyse['Indexes'] = 'NYSE'
df_nasdaq['Indexes'] = 'NASDAQ'
print("뉴욕증권거래소:",df_nyse.shape)
print("나스닥:",df_nasdaq.shape)

뉴욕증권거래소: (3117, 5)
나스닥: (4247, 5)


In [ ]:
#데이터 합치기
df = pd.concat([df_nyse, df_nasdaq])
#symbol 중복되는 데이터 제거
df = df.drop_duplicates('Symbol')

print(df.shape)
df.head(10)

(7355, 5)


,Symbol,Name,Industry,IndustryCode,Indexes
0,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체,57101010,NYSE
1,V,Visa Inc,온라인 서비스,57201030,NYSE
2,UNH,UnitedHealth Group Inc,의료 관리,56102020,NYSE
3,LLY,Eli Lilly and Co,제약,56201040,NYSE
4,BRK.B,Berkshire Hathaway Inc Class B,소비재 대기업,54401010,NYSE
5,XOM,Exxon Mobil Corp,"오일, 가스 정제 및 마케팅",50102030,NYSE
6,JNJ,Johnson & Johnson,제약,56201040,NYSE
7,JPM,JPMorgan Chase & Co,은행,55101010,NYSE
8,WMT,Walmart Inc,식품 소매 및 유통,54301020,NYSE
9,MA,Mastercard Inc,온라인 서비스,57201030,NYSE


In [ ]:
#용량 문제로 인해 100개만 잘라서 연습
df100 = df.head(100)
df100

,Symbol,Name,Industry,IndustryCode,Indexes
0,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체,57101010,NYSE
1,V,Visa Inc,온라인 서비스,57201030,NYSE
2,UNH,UnitedHealth Group Inc,의료 관리,56102020,NYSE
3,LLY,Eli Lilly and Co,제약,56201040,NYSE
4,BRK.B,Berkshire Hathaway Inc Class B,소비재 대기업,54401010,NYSE
...,...,...,...,...,...
95,BSX,Boston Scientific Corp,첨단 의료 장비 및 기술,56101010,NYSE
96,FI,Fiserv Inc,경영 지원 서비스,52203040,NYSE
97,ITW,Illinois Tool Works Inc,소비재 대기업,54401010,NYSE
98,SO,Southern Co,전력 유틸리티,59101010,NYSE


In [ ]:
#Symbol값만 추출해서 리스트로 만들기
df100_symbols = df100['Symbol'].tolist()

#데이터프레임 초기화
result_df = pd.DataFrame

for symbol in df100_symbols:
  try:
    df = fdr.DataReader(symbol)

    df.columns = [symbol+'_'+col for col in df.columns]

    if result_df.empty:
      result_df = df
    else:
      result_df = pd.merge(result_df, df, how='outer', left_index=True, right_index=True)
  except:
    #Yahoo Finance에서 BRK.B라는 심볼(Symbol)을 찾을 수 없어 발생하는 오류
    #HTTP Error 404: Not Found  - symbol "BRK.B"not found or invalid periods
    print(f'{symbol}의 데이터를 가져오지 못함')

result_df.to_csv('stock100_data.csv')


HTTP Error 404: Not Found  - symbol "BRK.B"not found or invalid periods
HTTP Error 404: Not Found  - symbol "BRK.A"not found or invalid periods
HTTP Error 404: Not Found  - symbol "UN"not found or invalid periods


In [ ]:
result_df.head(10)

In [ ]:
#열컬럼에 전부 NaN이 들어가면 drop
result_df = result_df.dropna(axis=1, how='all')
result_df

In [ ]:
#2003년부터 최근까지 데이터만
result_df2003 = result_df[result_df.index >= '2003-01-01']
result_df2003

,TSM_Open,TSM_High,TSM_Low,TSM_Close,TSM_Adj Close,TSM_Volume,V_Open,V_High,V_Low,V_Close,...,SO_Low,SO_Close,SO_Adj Close,SO_Volume,ENB_Open,ENB_High,ENB_Low,ENB_Close,ENB_Adj Close,ENB_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2003-01-02,5.342996,5.470032,5.260796,5.440141,2.803190,6774506.0,NaN,NaN,NaN,NaN,...,28.540001,28.980000,11.336046,1598300.0,6.835000,6.920000,6.800000,6.907500,2.970340,88800.0
2003-01-03,5.447614,5.612014,5.447614,5.567178,2.868649,5766574.0,NaN,NaN,NaN,NaN,...,28.950001,29.110001,11.386899,2048600.0,6.937500,6.992500,6.930000,6.960000,2.992915,156800.0
2003-01-06,5.589596,5.791359,5.559705,5.716632,2.945659,7918400.0,NaN,NaN,NaN,NaN,...,29.150000,30.809999,12.051884,6625600.0,6.975000,7.090000,6.970000,7.022500,3.019792,157200.0
2003-01-07,5.776414,5.836196,5.679268,5.739050,2.957212,7438789.0,NaN,NaN,NaN,NaN,...,29.430000,29.719999,11.625509,3724300.0,7.015000,7.045000,6.995000,7.010000,3.014417,158800.0
2003-01-08,5.679268,5.686741,5.477505,5.492450,2.830143,5619907.0,NaN,NaN,NaN,NaN,...,29.370001,29.570000,11.566836,1938900.0,7.020000,7.062500,6.930000,6.955000,2.990764,351600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,102.019997,103.040001,100.089996,100.110001,100.110001,8560000.0,229.440002,230.100006,226.199997,226.300003,...,70.419998,71.730003,71.730003,3078300.0,36.180000,37.000000,36.160000,36.849998,36.849998,3631900.0
2023-06-27,101.150002,102.790001,100.019997,102.080002,102.080002,9732000.0,227.479996,228.610001,226.520004,227.339996,...,70.849998,71.120003,71.120003,4005800.0,36.820000,36.939999,36.520000,36.740002,36.740002,2164600.0
2023-06-28,100.500000,101.879997,100.220001,100.919998,100.919998,8160900.0,227.500000,228.059998,226.020004,227.960007,...,69.709999,69.870003,69.870003,3895000.0,36.520000,36.770000,36.349998,36.720001,36.720001,2288800.0


In [ ]:
import pandas as pd

# 기존 데이터프레임에서 고유한 주식 종목 기호를 찾습니다.
symbols = set([col.split('_', 1)[0] for col in result_df2003.columns if '_' in col])

# 각 주식 종목별로 별도의 데이터프레임을 생성하고 해당 주식 기호를 할당합니다.
dfs = []
for symbol in symbols:
    # 모든 피쳐에 대해 동일한 처리를 수행하기 위한 수정
    temp_df = result_df2003.filter(regex=f"^{symbol}_")

    # 원래 컬럼 이름에서 기호를 제거합니다.
    new_columns = {col: col.split('_', 1)[1] for col in temp_df.columns}
    temp_df.rename(columns=new_columns, inplace=True)

    temp_df['Symbol'] = symbol
    temp_df['Date'] = result_df2003.index
    dfs.append(temp_df)

# 별도의 데이터프레임을 하나로 결합합니다.
stock_data_new = pd.concat(dfs).reset_index(drop=True)
stock_data_new

In [ ]:
stock_data_new.to_csv('stock_data_new.csv')

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def preprocess_stock_data(data):
    # 결측값 처리: 결측값이 있는 경우 해당 행과 열을 제거합니다.
    data_prep = data.dropna(axis=0, how='any')

    # 데이터를 정규화 및 스케일링합니다. 여기서 스케일링은 Min-Max 스케일링을 사용하며 다른 스케일링 방법으로 변경할 수 있습니다.
    numerical_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']  # 대상 수치 피쳐 추가/제거 가능

    scaler = MinMaxScaler()
    data_prep[numerical_features] = scaler.fit_transform(data_prep[numerical_features])

    return data_prep



def create_timeseries_tensor(data, n_steps):
    X = []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps].values)
    return np.array(X)

stock_data_prep = preprocess_stock_data(stock_data_new)  # 정규화 및 스케일링 처리
X_stock = create_timeseries_tensor(stock_data_prep, 30)




In [ ]:
X_stock

array([[[0.03300462246798422, 0.03386635098784077, 0.03334092966835177,
         ..., 0.0017301231268920803, 'BA',
         Timestamp('2003-01-02 00:00:00')],
        [0.03413799518100792, 0.034401241406681785,
         0.034266697827295106, ..., 0.0018772549169528672, 'BA',
         Timestamp('2003-01-03 00:00:00')],
        [0.03408647917618021, 0.034524679477073494, 0.03433951113509212,
         ..., 0.0018210921450127664, 'BA',
         Timestamp('2003-01-06 00:00:00')],
        ...,
        [0.0301917923477888, 0.03035871300547725, 0.029773077586298137,
         ..., 0.0022272737017290556, 'BA',
         Timestamp('2003-02-11 00:00:00')],
        [0.029707531186865, 0.02972095924236555, 0.02955463870309721,
         ..., 0.0020577257109664865, 'BA',
         Timestamp('2003-02-12 00:00:00')],
        [0.029357215347719786, 0.029638668566949, 0.029076150211669583,
         ..., 0.002336990814982141, 'BA',
         Timestamp('2003-02-13 00:00:00')]],

       [[0.03413799518100792, 0

##워렌버핏포트폴리오 가져오기 (2022 2분기)

In [ ]:
pip install finsec

In [ ]:
#pip install finsec

import finsec
# 버크셔헤서웨이(워렌버핏) 포트폴리오 0001067983
filing = finsec.Filing('0001067983')

#분기 설정
q3_cover_page, q3_holdings_table, q3_simplified_holdings_table = filing.get_a_13f_filing("Q3-2022")

df_table = q3_simplified_holdings_table

sum_of_value = df_table['Holding value'].sum()

df_table['percentage'] = round(df_table['Holding value']/sum_of_value * 100, 1)

# Name of issuer를 모두 소문자로 변환
df_table['Name of issuer'] = df_table['Name of issuer'].str.lower()

df_table.sort_values(by = ['percentage'], ascending = False, inplace = True)
df_table.reset_index(inplace=True, drop = True)
df_table.drop(['Title of class', 'Share or principal type','Share or principal amount count', 'Holding value','CUSIP'], axis = 1, inplace =True)
df_table

,Name of issuer,percentage
0,apple inc,41.8
1,bank amer corp,10.3
2,chevron corp new,8.0
3,coca cola co,7.6
4,american express co,6.9
5,occidental pete corp,4.0
6,kraft heinz co,3.7
7,moodys corp,2.0
8,activision blizzard inc,1.5
9,taiwan semiconductor mfg ltd,1.4


In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf